<img style="margin-left:auto; margin-right:0px" src="./images/projet2-notebook-header.gif">

```Date: 24/10/2020
 Auteur: Adrien Loridan 
 Courriel : adrien.loridan@gmail.com
 Version : 0.2```

# Mission
L'agence "Santé publique France" a lancé un appel à projets pour trouver des idées innovantes d’applications en lien avec l'alimentation.  
Nous allons y participer et proposer une idée d’application.

Il est demandé expressément d'utiliser les données provenant d'[Open Food Facts](https://fr.openfoodfacts.org/).
Les champs sont séparés en quatre sections :

1. Les informations générales sur la fiche du produit : nom, date de modification, etc.
2. Un ensemble de tags : catégorie du produit, localisation, origine, etc.
3. Les ingrédients composant les produits et leurs additifs éventuels.
4. Des informations nutritionnelles : quantité en grammes d’un nutriment pour 100 grammes du produit.

# Import des dépendances et des données
Le [fichier CSV](https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv) mise à disposition est d'une taille importante d'environ 3Go.  
Le fichier utilise l'encodage Unicode UTF-8. Le caractère de séparation des champs est <tab> (tabulation).   
Nous commençons donc par importer numpy et pandas puis nous chargeons en mémoire les données.

In [1]:
import numpy as np
import pandas as pd

filenamePath = 'en.openfoodfacts.org.products.csv'

dataframe = pd.read_csv(filenamePath, sep='\t')

C:\Users\adrie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,12,18,19,20,21,25,26,27,29,50,62,171) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Le chargement des données retourne un avertissement sur la présence de type différent dans une même colonne.  
Ceci est certainement dû au fait que le dataset provient d'une application n'ayant pas mis en place de contraintes sur les entrées utilisateurs.

# Un premier regard
Je vais donc prendre connaissance du véritable contenu et de la qualité de mes données.

In [2]:
dataframe.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1486047 entries, 0 to 1486046
Data columns (total 182 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   code                                        object 
 1   url                                         object 
 2   creator                                     object 
 3   created_t                                   int64  
 4   created_datetime                            object 
 5   last_modified_t                             int64  
 6   last_modified_datetime                      object 
 7   product_name                                object 
 8   generic_name                                object 
 9   quantity                                    object 
 10  packaging                                   object 
 11  packaging_tags                              object 
 12  packaging_text                              object 
 13  brands                    

Nous constatons comme prévu que nous avons une liste de produits alimentaires avec un ensemble de caractéristiques.  
Une explication plus détaillée est donnée sur [openfoodfacts-data-fields](https://world.openfoodfacts.org/data/data-fields.txt).

In [3]:
pd.options.display.max_columns = 182
dataframe.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,375.0,1569.0,NaN,7.0,3.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.1,15.0,NaN,NaN,NaN,NaN

On peut constater la présence de nombreux champs manquants (NaN).

# Nettoyage des données
Avant de réaliser quelques analyses statistiques sur nos données, il va nous falloir les nettoyer un minimum.   

### Suppression des doublons
Ma première approche consiste à identifier et à retirer les produits qui seraient en double.  
Vu que le code-barre est censé être unique, on va s'en servir comme identifiant.

In [4]:
dataframe2 = dataframe.drop_duplicates(subset='code')
number_deleted_rows = dataframe.shape[0] - dataframe2.shape[0]
print("On a supprimé {} lignes".format(number_deleted_rows)+" soit {:.2f}".format(number_deleted_rows/dataframe.shape[0]*100)+'%')

On a supprimé 326 lignes soit 0.02%


### Identifier et supprimer les colonnes peu fiables
Nous allons d'abord calculer le pourcentage de données manquantes pour chaques colonnes.

In [5]:
percentageNanbyColumns = dataframe2.isnull().sum(axis = 0) / dataframe2.shape[0] * 100.00
percentageNanbyColumns

code                   0.000000
url                    0.000000
creator                0.000269
created_t              0.000000
created_datetime       0.000000
                        ...    
choline_100g          99.997442
phylloquinone_100g    99.884837
beta-glucan_100g      99.998385
inositol_100g         99.997038
carnitine_100g        99.998654
Length: 182, dtype: float64

On affiche les colonnes qui ont été remplies à au moins 50 %

In [6]:
percentageNanbyColumns[percentageNanbyColumns<50]

code                       0.000000
url                        0.000000
creator                    0.000269
created_t                  0.000000
created_datetime           0.000000
last_modified_t            0.000000
last_modified_datetime     0.000000
product_name               4.581883
brands                    44.468712
brands_tags               44.472347
categories                49.264431
categories_tags           49.264431
categories_en             49.264431
countries                  0.343739
countries_tags             0.344008
countries_en               0.344008
pnns_groups_1              0.969698
pnns_groups_2              0.019788
states                     0.000000
states_tags                0.000000
states_en                  0.000000
main_category             49.264431
main_category_en          49.264431
image_url                 28.925889
image_small_url           28.925889
energy-kcal_100g          24.142487
energy_100g               19.555556
fat_100g                  20

Je vais me concentrer sur ces colonnes afin de tirer quelques hypothéses.   
Je supprime donc les colonnes de mon dataframe qui ne sont pas dans cette liste.

In [7]:
dataframe3 = dataframe2.drop(percentageNanbyColumns[percentageNanbyColumns>50].index,axis=1)

### Traitement des données manquantes